# Test DeepWay, a DNN able to Detect Waypoints for Autonomous Navigation in Row based Crops

Test DeepWay to automatically detect waypoints from a occupancy grid map of a row based crop. The waypoints can be used to generate aa road map for a UGV. That combined with a local motion planner and data fusion localization methodologies can provide a robust and complete autonomous navigation in vineyards. 

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
# import some libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import glob
import cv2
from tqdm.notebook import tqdm
from utils.visualization import plotData, plotHistory, plotDataRes, plotImages
from utils.tools_net import waypointProxSup, interpret, deepPathLoss, AP
from utils.tools import load_config, resizeAddCorrection
from utils.deep_way_net import build_deepway

In [ ]:
# select a GPU and set memory growth 
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

In [ ]:
# important paths and names
PATH_DIR = os.path.abspath('.')

TEST_DATA_PATH = os.path.join(PATH_DIR, 'dataset/test')
SATELLITE_DATA_PATH = os.path.join(PATH_DIR, 'dataset/satellite')
PATH_WEIGHTS = os.path.join(PATH_DIR, 'bin')

#name_model = 'deep_way_pretrained.h5'
name_model = 'deep_way.h5'



waypoint_file_name = 'waypoints.csv'
config_file = 'utils/config.json'

In [ ]:
# important config
config = load_config(config_file)

# 1.0 Import the Test Dataset

In [ ]:
def load_dataset_test(df, path_test):
    img_list = glob.glob(os.path.join(path_test, '*.png'))
    img_list = sorted(img_list,key=lambda s: int(s.split('/')[-1][3:-4])) #sort by name
    data_n = len(img_list)
    MASK_DIM = config['MASK_DIM'] 
    X = np.empty((data_n, MASK_DIM, MASK_DIM), dtype='float32')
    y = np.empty((data_n, MASK_DIM, MASK_DIM), dtype='float32')
    for index,img in tqdm(enumerate(img_list),total=data_n):
        name = img.split('/')[-1][:-4]
        mask = cv2.bitwise_not(cv2.imread(img, cv2.IMREAD_GRAYSCALE)) # open grayscale and invert 255
        points = df.loc[df['N_img'] == name].to_numpy()[:,1:].astype('uint32')
        points_x = points[:,0]
        points_y = points[:,1]
        mask_points = np.zeros((MASK_DIM,MASK_DIM))
        mask_points[points_y, points_x] = config['WAYP_VALUE']
        
        X[index,:,:] = mask
        y[index,:,:] = mask_points
    return X, resizeAddCorrection(y, config['WAYP_VALUE'], config['K'])

In [ ]:
#img_folder = SATELLITE_DATA_PATH  #for satellite test set
img_folder = TEST_DATA_PATH       #for artificial test set

df_waypoints_test = pd.read_csv(os.path.join(img_folder, waypoint_file_name))

In [ ]:
X_test, y_test = load_dataset_test(df_waypoints_test, img_folder)

# 2.0 Import DeepWay

In [ ]:
# load network
deepway_net = build_deepway(name_model, config['FILTERS'], config['KERNEL_SIZE'],
                            config['N'], config['MASK_DIM'])
deepway_net.load_weights(os.path.join(PATH_WEIGHTS, name_model))

In [ ]:
name_model

# 3.0 Test some Random Predictions

In [ ]:
index = 6 # choose a map
dist_thresh = 8 # waypoint suppression
conf_thresh = 0.3

pred = interpret(deepway_net.predict(X_test[index:index+1]),
                                 conf_thresh = conf_thresh, dist_thresh = dist_thresh,
                         waypoint_prox_sup=True, K=config['K'], MASK_DIM=config['MASK_DIM'])

plotData(X_test[index], pred[0])

# 4.0 Plot Multiple Images

In [ ]:
index = 20
n = 3
dist_thresh = 8 # waypoint suppression
conf_thresh = 0.3

y_pred = interpret(deepway_net.predict(X_test[index:index+n]),
                                 conf_thresh = conf_thresh, dist_thresh = dist_thresh, waypoint_prox_sup=True,
                               K=config['K'], MASK_DIM=config['MASK_DIM'])

plotImages(X_test[index:index+n], y_pred, n)

# 5.0 Compute AP

In [ ]:
DIST_RANGE_PERC = 2 # minimum interow distance
DIST_TRESH = 8 # waypoint suppression

rec_tot, prec_tot = AP(X_test, y_test, deepway_net, True, 
                       DIST_RANGE = DIST_RANGE_PERC, K=config['K'], MASK_DIM=config['MASK_DIM'], 
                       WAYP_VALUE = config['WAYP_VALUE'], dist_thresh = DIST_TRESH)

In [ ]:
plt.scatter(rec_tot, prec_tot, c='r', s=15)
plt.plot(rec_tot, prec_tot, c='r')
plt.title(f"AP@{DIST_RANGE_PERC}: {np.mean(prec_tot):.4f}", )
plt.xlabel('Recall')
plt.ylabel('Precision')
#plt.xlim((0.8260,0.8275))
plt.grid()
plt.show()